# NOTEBOOK PARTE 2: MODELADO TRADUCTOR PT -> ES

# CONFIGURACIÓN DEL ENTORNO

## Librerías

In [1]:
import os
import random
import torch
from tqdm.auto import tqdm
from datasets import Dataset  
from datasets import DatasetDict 
from datasets import load_dataset
from transformers import set_seed   
from transformers import AutoTokenizer              

## Reproducibilidad

In [2]:
SEED = 333
set_seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

## Selección dispositivo

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


## Path de los datos

In [4]:
DATA_DIR = "./data"
PT_FILE  = os.path.join(DATA_DIR, "clean_corpus_pt.txt")
ES_FILE  = os.path.join(DATA_DIR, "clean_corpus_es.txt")

assert os.path.isfile(PT_FILE), f"No encontrado: {PT_FILE}"
assert os.path.isfile(ES_FILE), f"No encontrado: {ES_FILE}"
print(f"Archivos disponibles:\n   • {PT_FILE}\n   • {ES_FILE}")

Archivos disponibles:
   • ./data\clean_corpus_pt.txt
   • ./data\clean_corpus_es.txt


# Carga del corpus preprocesado

In [5]:
# Carga del corpus limpio desde JSON
# Ruta al archivo JSON generado en Notebook 1
JSON_FILE = os.path.join(DATA_DIR, "clean_corpus_pt_es.json")

# Cargar el Dataset de Hugging Face directamente desde JSON
corpus_dataset = load_dataset(
    "json",
    data_files=JSON_FILE,
    field=None,          # cada objeto JSON ya es {"pt": ..., "es": ...}
    split="train"        # se carga todo como un único split "train"
)

# Verificar número de ejemplos y columnas
print(f"Pares cargados: {len(corpus_dataset)}")
print("Columnas disponibles:", corpus_dataset.column_names)

Pares cargados: 2480081
Columnas disponibles: ['pt', 'es']


# División del dataset

In [6]:
# Definir proporciones
TEST_SIZE = 0.2  # 20% para test

# Usar el método incorporado de HF Datasets
splits = corpus_dataset.train_test_split(test_size=TEST_SIZE, seed=SEED)

# Renombrar las particiones
dataset_dict = DatasetDict({
    "train": splits["train"],
    "test" : splits["test"]
})

# Comprobar tamaños
print(f"Entrenamiento: {len(dataset_dict['train'])} ejemplos")
print(f"     Prueba : {len(dataset_dict['test'])} ejemplos")


Entrenamiento: 1984064 ejemplos
     Prueba : 496017 ejemplos


# Tokenización y preparación de los datos de entrada

In [7]:
# Carga del tokenizer M2M100
MODEL_NAME = "facebook/m2m100_418M"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)

# Configurar idiomas de origen y destino
tokenizer.src_lang  = "pt"
tokenizer.tgt_lang  = "es"
forced_bos_token_id = tokenizer.get_lang_id("es")

In [8]:
# Función de tokenización 
MAX_LENGTH = 128

def tokenize_fn(batch):
    # Tokenizar simultáneamente entrada y etiqueta
    outputs = tokenizer(
        batch["pt"],
        text_target=batch["es"],
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True,
        return_attention_mask=True
    )
    # outputs ya contiene 'input_ids', 'attention_mask' y 'labels' correctos,
    # donde 'labels' tiene padding token id (no -100 aún).
    # Convertimos padding en -100
    labels = [
        [(token_id if token_id != tokenizer.pad_token_id else -100)
         for token_id in seq]
        for seq in outputs["labels"]
    ]
    outputs["labels"] = labels
    return outputs

In [9]:
# Aplicar tokenización
tokenized_datasets = dataset_dict.map(
    tokenize_fn,
    batched=True,
    remove_columns=["pt", "es"],
    desc="Tokenizando con M2M100 (text_target)",
    load_from_cache_file=True
)

print("Columnas tras tokenización:", tokenized_datasets["train"].column_names)
print(f"Ejemplos train: {len(tokenized_datasets['train'])}")
print(f"Ejemplos test : {len(tokenized_datasets['test'])}")

Columnas tras tokenización: ['input_ids', 'attention_mask', 'labels']
Ejemplos train: 1984064
Ejemplos test : 496017


In [10]:
# Revisamos ejemplos de tokenización
from datasets import load_from_disk

# 1) Recarga el dataset tokenizado y el raw split de entrenamiento
tokenized = tokenized_datasets["train"]
raw_split = dataset_dict["train"]   # contiene {"pt","es"} originales

# 2) Elige 3 índices dentro del rango de train
for idx in random.sample(range(len(tokenized)), 3):
    raw_item  = raw_split[idx]
    tok_item  = tokenized[idx]
    
    pt_orig   = raw_item["pt"]
    es_ref    = raw_item["es"]
    # Decodifica solo los input_ids (entrada PT)
    pt_tok    = tokenizer.decode(tok_item["input_ids"], skip_special_tokens=True)
    # Decodifica los labels (restaurando pad_token_id)
    labels = [tid if tid != -100 else tokenizer.pad_token_id
              for tid in tok_item["labels"]]
    es_tok = tokenizer.decode(labels, skip_special_tokens=True)
    
    print(f"--- Ejemplo #{idx} ---")
    print(f"PT original : {pt_orig}")
    print(f"PT tokenizado: {pt_tok}")
    print(f"ES referencia: {es_ref}")
    print(f"ES tokenizado: {es_tok}\n")

--- Ejemplo #1163574 ---
PT original : Recolha e comparação de informação e criação de redes de especialistas e de institutos;
PT tokenizado: Recolha e comparação de informação e criação de redes de especialistas e de institutos;
ES referencia: Recopilación y cotejo de información y creación de redes de especialistas e institutos.
ES tokenizado: Recopilación y cotejo de información y creación de redes de especialistas e institutos.

--- Ejemplo #735697 ---
PT original : Produz valor acrescentado em relação às actividades que visam promover a igualdade a nível nacional.
PT tokenizado: Produz valor acrescentado em relação às actividades que visam promover a igualdade a nível nacional.
ES referencia: Proporcionará un valor añadido a los trabajos en materia de igualdad realizados a escala nacional.
ES tokenizado: Proporcionará un valor añadido a los trabajos en materia de igualdad realizados a escala nacional.

--- Ejemplo #740577 ---
PT original : Esta dotação destina-se a cobrir as despe

Los ejemplos confirman que la tokenización:
- Respeta la segmentación de sub-palabras en ambos idiomas sin introducir ni omitir caracteres: por ejemplo, el corchete y punto y coma en el tercer ejemplo aparecieron intactos.
- Aplica truncamiento solo cuando sobrepasa 128 tokens, pero en estos casos las oraciones son más cortas, así que no hubo cortes.
- Reproduce fielmente los labels, ya que al decodificar las labels (con padding convertido a –100) recuperamos exactamente la referencia en español sin tokens extra.

In [17]:
# Guardar el dataset tokenizado en disco
# Construir un nombre de carpeta que incluya modelo y longitud máxima
save_dir = os.path.join(
    DATA_DIR,
    f"tokenized_{MODEL_NAME.replace('/', '_')}_len{MAX_LENGTH}"
)
tokenized_datasets.save_to_disk(save_dir)
print(f"Tokenized datasets guardados en: {save_dir}")

# Recargar con:
# from datasets import load_from_disk
# tokenized_datasets = load_from_disk(save_dir)

Saving the dataset (0/7 shards):   0%|          | 0/1984064 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/496017 [00:00<?, ? examples/s]

Tokenized datasets guardados en: ./data\tokenized_facebook_m2m100_418M_len128


Comentarios sobre la elección del tokenizer y parámetros

- **Modelo M2M100 multilingüe**: se seleccinó `facebook/m2m100_418M` porque soporta de forma nativa la traducción directa PT→ES dentro de un único paso, evitando la complejidad de encadenar modelos. 

- **Configuración de idioma**: al establecer `tokenizer.src_lang = "pt"` y `tokenizer.tgt_lang = "es"`, se garantiza que el tokenizador segmente y represente los tokens correctamente según las características de cada idioma, mejorando la calidad de la generación.

- **Uso de `text_target`**: evitamos el método obsoleto `as_target_tokenizer` pasando el texto de destino directamente con `text_target=batch["es"]`. Esto unifica la tokenización de entrada y salida en una sola llamada, simplificando el flujo y eliminando warnings.

- **Longitud máxima (`MAX_LENGTH=128`)**: equilibramos cobertura de contexto y eficiencia computacional. Un límite de 128 tokens cubre la gran mayoría de oraciones sin sobrepasar la memoria de la GPU, reduciendo la necesidad de truncamiento agresivo.

- **Padding y `labels`**: aplicamos `padding="max_length"` para batch uniformes y reemplazamos los `pad_token_id` en las etiquetas por `-100` para que no contribuyan a la función de pérdida, siguiendo la convención de Hugging Face para entrenamiento Seq2Seq.
